# Kernel Density Estimate

Alessandro Samuel-Rosa, Taciara Zborowski Horst

2025-07-06

License: MIT

In [161]:
import ee
import geemap

try:
    # Initialize the Earth Engine library.
    # Authenticate and initialize GEE if you are running this locally.
    # In some environments like Google Colab, ee.Authenticate() might not be needed.
    # ee.Authenticate()
    ee.Initialize(project='mapbiomas-solos-workspace') # Your project ID
except Exception as e:
    print(f"Earth Engine initialization failed: {e}")
    # Fallback for environments where initialization might fail but a default project exists
    try:
        ee.Initialize()
    except Exception as e_fallback:
        print(f"Fallback Earth Engine initialization also failed: {e_fallback}")
        # Exit or handle the error as appropriate for your application
        exit()

In [162]:
# 1. Define the input data path
fc_dir = 'projects/mapbiomas-workspace/SOLOS/AMOSTRAS/MATRIZES/granulometry/'
fc_name = 'matriz-v010_0_10cm'
soil_samples_path = f"{fc_dir}{fc_name}"

In [163]:
# 2. Load the datasets
try:
    # Load the soil sampling points as a FeatureCollection
    soil_samples = ee.FeatureCollection(soil_samples_path)
    # Load Brazil boundary for clipping the final result
    brazil_boundary = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME', 'Brazil'))
except ee.EEException as e:
    print(f"Error loading data from Earth Engine: {e}")
    exit()

In [164]:
# 3. Define projection and scale for the analysis
# We define these manually since we are not using a reference image.
# A standard projection like WGS84 (EPSG:4326) is suitable.
target_crs = 'EPSG:4326'
# A 1km scale is efficient for a country-wide density map.
calculation_scale = 1000 # meters

In [165]:
# 4. Compute the Kernel Density Estimate (KDE)
# To fix the binary output issue, we use reduceToImage to rasterize the points,
# which is more robust than the paint() method for this task.

# Define the desired geographical radius for the kernel.
kernel_radius_meters = 50000 # 50km

# Calculate the kernel radius in pixels for our chosen calculation scale.
kernel_radius_pixels = kernel_radius_meters / calculation_scale # 50000m / 1000m/pixel = 50 pixels

# Add a property to each feature to be used as the 'weight' in the rasterization.
soil_samples_with_weight = soil_samples.map(lambda f: f.set('density', 1))

# Create an image of point counts per pixel at the coarse resolution.
# This rasterizes the points into a grid. We use sum() in case multiple points fall in one pixel.
image_with_points = soil_samples_with_weight.reduceToImage(
    properties=['density'],
    reducer=ee.Reducer.sum()
).reproject(
    crs=target_crs,
    scale=calculation_scale
).unmask(0) # Ensure empty pixels are 0, which is crucial for the kernel operation.


# Create a Gaussian kernel using pixel units.
# The radius of 50 pixels is efficient and well within GEE's limits.
# normalize=True is the correct setting to create a probability density surface.
kernel = ee.Kernel.gaussian(
    radius=kernel_radius_pixels,
    units='pixels',
    normalize=True
)

# Apply the kernel using convolve(), which applies the kernel to each pixel's neighborhood.
kde_image_unclipped = image_with_points.convolve(kernel)


# Clip the final KDE image to the boundary of Brazil.
kde_image = kde_image_unclipped.clip(brazil_boundary)

In [166]:
# 5. Define visualization parameters for the KDE map
# A fixed 'max' value is used instead of dynamic calculation for better performance
# and consistency across a large area like Brazil.
# This value is a guess and may need to be adjusted to best visualize
# the density variations in your specific dataset. If the map appears
# saturated (mostly red), increase this value. If it's too faint, decrease it.
fixed_max_value = 0.0005

# We'll use a color palette that goes from transparent/light for low density
# to a strong color for high density.
kde_vis_params = {
    'min': 0,
    'max': fixed_max_value,
    'palette': ['#FFFFFF00', 'yellow', 'orange', 'red', 'darkred']
}

In [167]:
# 6. Create and display the map
# Initialize a geemap Map object
m = geemap.Map(center=[-14, -54], zoom=4) # Center on Brazil

# Add the final KDE layer to the map
m.addLayer(kde_image, kde_vis_params, 'Soil Sample Density (KDE)')

# Add the original soil sample points for reference
m.addLayer(soil_samples, {'color': 'blue'}, 'Soil Sample Points', False) # Initially off

# Add the Brazil boundary for context
m.addLayer(brazil_boundary, {}, 'Brazil Boundary', False) # Initially off

# Add a layer control to toggle layers on and off
m.addLayerControl()

# Display the map
m

Map(center=[-14, -54], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…